# First learning attempts
### using Beethoven dataset
* 29 pieces + transpositions across 2 octaves
* ~70h of music (2.7h per transposition)
* 0.025s resolution (40fps)

In [ ]:
IGNORE_NOTE_VELOCITY = True

# loading data files names
import os

path = '.\\datasets\\beethoven\\'
file_names = os.listdir(path)
file_names = list(filter(lambda fn: '.npz' in fn or '.npy' in fn or '.csv' in fn, file_names))
assert len(file_names) > 0, 'Data not found'

f'Found {len(file_names)} files'

In [ ]:
# loading data files
from midi_numpy.common import read_numpy_midi
file_paths = [f'{path}{fn}' for fn in file_names]

from random import choice
def load_tracks(n):
    print('loading tracks')
    sampled_file_paths = [choice(file_paths) for _ in range(n)]
    return [read_numpy_midi(fp) for fp in sampled_file_paths]

### Creating x and y's

In [ ]:
def create_x_y(tracks):
    print('creating x/y')
    data_x = [t[:-1] for t in tracks]
    data_y = [t[1:] for t in tracks]
    if IGNORE_NOTE_VELOCITY:
        data_x = [dx[:, :128] for dx in data_x]
        data_y = [dy[:, :128] for dy in data_y]
    return data_x, data_y

### Processing data

In [ ]:
# splitting data into chunks (sequences of equal length)
import numpy as np
CHUNK_LENGTH = 200 # equals to 5s at 0.025s frames

def chunkify(data_x, data_y, chunks_per_track_mult=10):
    print('chunking')
    # for each track picks (10*len(dx)//CHUNK_LENGTH+1) random sequences
    data_x_y = [
        (dx[split_point:split_point + CHUNK_LENGTH], dy[split_point:split_point + CHUNK_LENGTH]) 
            for dx, dy in zip(data_x, data_y)
            for _ in range(chunks_per_track_mult * len(dx) // CHUNK_LENGTH + 1)
            for split_point in [np.random.randint(len(dx) - 1)]
    ]
    return list(zip(*data_x_y))

In [ ]:
# pad smaller chunks to CHUNK_SIZE
def pad_chunk_sequence(chunk, goal_seq):
    d_len = goal_seq - chunk.shape[0]
    npad = ((0, d_len), (0, 0))
    return np.pad(chunk, npad, 'constant')
    
def padify(data_x, data_y):
    print('padding')
    data_x_res = [pad_chunk_sequence(chunk, CHUNK_LENGTH) for chunk in data_x]
    data_y_res = [pad_chunk_sequence(chunk, CHUNK_LENGTH) for chunk in data_y]
    return data_x_res, data_y_res

In [ ]:
# convert list of matrices to highier dim matrices
def stackify(data_x, data_y):
    print('stacking')
    data_x_res = np.stack(data_x)
    data_y_res = np.stack(data_y)
    return data_x_res, data_y_res

In [ ]:
# dataset generator
def data_gen(batch_size, track_count=25):
    # x data shape should be [batch_size, sequence_len, input_dim]
    # since training will be in many-to-many mode, y has same shape    
    while True:
        print('reloading data')
        data_x, data_y = None, None
        data_x, data_y = stackify(*padify(*chunkify(*create_x_y(load_tracks(track_count)))))
        n_samples = len(data_x)
        print(f'loaded {n_samples} samples')
        for _ in range(10 * n_samples):
            indices = np.random.randint(0, n_samples, batch_size)
            yield data_x[indices], data_y[indices]            

## Setting up model

In [ ]:
from tensorflow import keras as K

INPUT_SIZE = 128 if IGNORE_NOTE_VELOCITY else 256
HIDDEN_SIZE = 512
OUTPUT_SIZE = INPUT_SIZE

BATCH_SIZE = 16
SEQUENCE_LENGTH = CHUNK_LENGTH

INPUT_SHAPE = (None, INPUT_SIZE)
# could be INPUT_SHAPE = (SEQUENCE_LENGTH, INPUT_SIZE)
# however predicting would have to have same seq length

In [ ]:
model = K.models.Sequential([
    K.layers.LSTM(HIDDEN_SIZE, input_shape=INPUT_SHAPE, return_sequences=True),
    K.layers.Dense(OUTPUT_SIZE, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['binary_accuracy', K.metrics.FalsePositives(), K.metrics.FalseNegatives()]
)

In [ ]:
# or load saved model
base_path = ''
file_name = 'beth_notransp_randchunk_bcr_512_22epochs_90.0m.h5'
model = K.models.load_model(base_path + file_name)

In [ ]:
# pre running operations
# some stat data accumultors for re-running model
from time import time
epochs_elapsed = 0
minutes_elapsed = 0
gen = data_gen(BATCH_SIZE)
test_gen = data_gen(BATCH_SIZE, 3)

### Running model

In [ ]:
EPOCHS = 1
STEPS_PER_EPOCH = 1000
start_time = time()

model.fit_generator(
    gen, 
    steps_per_epoch=STEPS_PER_EPOCH, 
    epochs=EPOCHS, 
    validation_data=test_gen, 
    validation_steps=100
)

minutes_elapsed += (time() - start_time) // 60
epochs_elapsed += EPOCHS

### Saving model

In [ ]:
base_path = ''
keywords = '_'.join(['beth', 'notransp', 'randchunk'])
file_name = f'{keywords}_{HIDDEN_SIZE}_{epochs_elapsed}epochs_{minutes_elapsed}m.h5'

K.models.save_model(model, base_path + file_name)